# Jet Energy Resolution

Jets are stochastic objects. The content of jets fluctuates quite a lot, and the content also depends on what actually caused the jet (uds quarks, gluons, etc). In addition, there are experimental limitations to the measurement of jets. Both of these aspects limit the accuracy to which we can measure the 4-momentum of a jet. This is called the jet resolution. If you have a group of single pions that have the same energy, the energy measured by CMS will not be exactly the same every time, but will typically follow a (roughly) Gaussian distribution with a mean and a width. The mean is corrected using the jet energy corrections. It is impossible to "correct" for all resolution effects on a jet-by-jet basis, although regression techniques can account for many effects.

As such, there will always be some experimental and theoretical uncertainty in the jet energy measurement, so this is the jet energy resolution. There is also jet angular resolution, and jet mass resolution. We will demonstrate how to apply the jet energy resolution, since that is applicable for all analyses that use jets. More information can be found at the jet resolution twiki and jet resolution software guide. The resolution is measured in data for different eta bins, and was approximately 10% with a 10% uncertainty for 7 TeV and 8 TeV data. For precision, it is important to use the correctly measured resolutions, but a reasonable calculation is to assume a flat 10% uncertainty for simplicity.

To perform this on `pat::Jets` in MC miniAOD, the syntax is:

```
smear = getJER(jet.eta(), 0) #JER nominal=0, up=+1, down=-1
smearUp = getJER(jet.eta(), 1) #JER nominal=0, up=+1, down=-1
smearDn = getJER(jet.eta(), -1) #JER nominal=0, up=+1, down=-1
recopt = jet.pt()
genpt = genJet.pt()
deltapt = (recopt-genpt)*(smear-1.0)
deltaptUp = (recopt-genpt)*(smearUp-1.0)
deltaptDn = (recopt-genpt)*(smearDn-1.0)
ptsmear = max(0.0, (recopt+deltapt)/recopt)
ptsmearUp = max(0.0, (recopt+deltaptUp)/recopt)
ptsmearDn = max(0.0, (recopt+deltaptDn)/recopt)
corr *= ptsmear
corrUp *= ptsmearUp
corrDn *= ptsmearDn
```

You can see that the smearing scales the difference between the reconstructed and truth-level jet $p_{\mathrm{T}}$s. The smearing value is taken from the function `getJER()`. 

Run the cell below to create histograms with the JER smearing applied. As usual, open `jmedas_make_histograms.py` again, and understand what the `getJER()` function does.  

In [ ]:
### RUN THIS CELL ONLY IF YOU ARE USING SWAN 
import os

##### REMEMBER TO MANUALLY COPY THE PROXY TO YOUR CERNBOX FOLDER AND TO MODIFY THE NEXT LINE
os.environ['X509_USER_PROXY'] = '/eos/home-X/Y/tmp/x509up_u0000'
if os.path.isfile(os.environ['X509_USER_PROXY']): pass
else: print("os.environ['X509_USER_PROXY'] ",os.environ['X509_USER_PROXY'])
os.environ['X509_CERT_DIR'] = '/cvmfs/cms.cern.ch/grid/etc/grid-security/certificates'
os.environ['X509_VOMS_DIR'] = '/cvmfs/cms.cern.ch/grid/etc/grid-security/vomsdir'

In [ ]:
%%bash
python $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_make_histograms.py --files=$CMSSW_BASE/src/Analysis/JMEDAS/data/MiniAODs/RunIIFall17MiniAODv2/ttjets.txt --outname=$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/ttjets_corr_smear.root --maxevents=2000 --maxFiles 10 --maxjets=2 --correctJets Fall17_17Nov2017_V32_MC --smearJets


...and run the following cell to plot the histograms. 

For our sample, which has a larger effect, jet energy correction uncertainty, or jet energy resolution uncertainty?


In [ ]:
import ROOT

f = ROOT.TFile("$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/ttjets_corr_smear.root")
h_ptAK4   = f.Get("h_ptAK4")
h_ptAK4Up  = f.Get("h_ptUpAK4")
h_ptAK4Down  = f.Get("h_ptDownAK4")
h_ptAK4Up.SetLineStyle(2)
h_ptAK4Up.SetLineColor(ROOT.kGreen+1) 
h_ptAK4Down.SetLineStyle(2)
h_ptAK4Down.SetLineColor(ROOT.kRed) 
h_ptAK4Up.SetLineWidth(2)
h_ptAK4Down.SetLineWidth(2)
h_ptAK4.SetLineWidth(2)

c = ROOT.TCanvas('c', 'c')

h_ptAK4.Draw()
h_ptAK4Up.Draw("same")
h_ptAK4Down.Draw("same")
h_ptAK4.GetXaxis().SetRangeUser(0, 400)

leg = ROOT.TLegend(0.45, 0.55, 0.75, 0.8)
leg.AddEntry(h_ptAK4, "Nominal JER", "l")
leg.AddEntry(h_ptAK4Up, "JER +1 #sigma", "l")
leg.AddEntry(h_ptAK4Down, "JER -1 #sigma", "l")
leg.SetLineWidth(0)
leg.SetFillColor(0)
leg.SetShadowColor(0)
leg.Draw()

c.Draw()

## Dijet resonance peaks
As a final exercise for the first part of this exercise, let's look at a simple dijet resonance peak. The following cell will produce histograms from an MC sample of Randall-Sundrum gravitons (RSGs) with m=3 TeV decaying to two quarks. The resulting signature is two high-$p_{\mathrm{T}}$ jets, with a truth-level invariant mass of 3 TeV. 

While the code is running, look at the script (scripts/jmedas_dijet.py). 

In [ ]:
%%bash
python $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_dijet.py --files=$CMSSW_BASE/src/Analysis/JMEDAS/data/MiniAODs/RunIIFall17MiniAODv2/rsgluon_qq_3000GeV.txt --outname=$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/rsgluon_qq_3000GeV.root --maxevents=4000 --maxjets=6
python $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_dijet.py --files=$CMSSW_BASE/src/Analysis/JMEDAS/data/MiniAODs/RunIIFall17MiniAODv2/rsgluon_qq_3000GeV.txt --outname=$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/rsgluon_qq_3000GeV_corr.root --maxevents=4000 --maxjets=6 --correctJets Fall17_17Nov2017_V32_MC
python $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_dijet.py --files=$CMSSW_BASE/src/Analysis/JMEDAS/data/MiniAODs/RunIIFall17MiniAODv2/rsgluon_qq_3000GeV.txt --outname=$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/rsgluon_qq_3000GeV_smear.root --maxevents=4000 --maxjets=6 --smearJets
python $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_dijet.py --files=$CMSSW_BASE/src/Analysis/JMEDAS/data/MiniAODs/RunIIFall17MiniAODv2/rsgluon_qq_3000GeV.txt --outname=$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/rsgluon_qq_3000GeV_corr_smear.root --maxevents=4000 --maxjets=6 --correctJets Fall17_17Nov2017_V32_MC --smearJets


...and then plot the resulting histograms.

In [ ]:
f_nominal = ROOT.TFile("$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/rsgluon_qq_3000GeV.root")
h_nominal = f_nominal.Get("h_dijet_mass")
h_nominal.SetName("h_dijet_mass_nominal")
h_nominal.SetDirectory(0)
h_gen = f_nominal.Get("h_dijet_genmass")
h_gen.SetDirectory(0)
f_nominal.Close()

f_corr = ROOT.TFile("$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/rsgluon_qq_3000GeV_corr.root")
h_corr = f_corr.Get("h_dijet_mass")
h_corr.SetName("h_dijet_mass_corr")
h_corr.SetDirectory(0)
f_corr.Close()

f_smear = ROOT.TFile("$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/rsgluon_qq_3000GeV_smear.root")
h_smear = f_smear.Get("h_dijet_mass")
h_smear.SetName("h_dijet_mass_smear")
h_smear.SetDirectory(0)
f_smear.Close()

f_corr_smear = ROOT.TFile("$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/rsgluon_qq_3000GeV_corr_smear.root")
h_corr_smear = f_corr_smear.Get("h_dijet_mass")
h_corr_smear.SetName("h_dijet_mass_corr_smear")
h_corr_smear.SetDirectory(0)
f_corr_smear.Close()

rebin = 4

c_dijet = ROOT.TCanvas("c_dijet", "Dijet mass (#Delta#eta<1.2)", 800, 600)
h_gen.SetLineColor(ROOT.kBlack)
h_gen.Rebin(rebin)
h_gen.Draw("hist")

h_nominal.SetLineColor(ROOT.kRed-1)
h_nominal.SetLineStyle(3)
h_nominal.Rebin(rebin)
h_nominal.Draw("hist same")

h_corr.SetLineColor(ROOT.kBlue+2)
h_corr.SetLineStyle(4)
h_corr.Rebin(rebin)
h_corr.Draw("hist same")

h_smear.SetLineColor(ROOT.kGreen-1)
h_smear.SetLineStyle(5)
h_smear.Rebin(rebin)
h_smear.Draw("hist same")

h_corr_smear.SetLineColor(ROOT.kMagenta)
h_corr_smear.SetLineStyle(2)
h_corr_smear.Rebin(rebin)
h_corr_smear.Draw("hist same")

l_dijet = ROOT.TLegend(0.22, 0.5, 0.4, 0.8)
l_dijet.SetFillColor(0)
l_dijet.SetFillStyle(0)
l_dijet.SetBorderSize(0)
l_dijet.AddEntry(h_gen, "Gen jets", "l")
l_dijet.AddEntry(h_nominal, "Uncorrected", "pl")
l_dijet.AddEntry(h_corr, "Corrected", "pl")
l_dijet.AddEntry(h_smear, "Smeared", "pl")
l_dijet.AddEntry(h_corr_smear, "Corr+smear", "pl")
l_dijet.Draw()

c_dijet.Draw()


# Conclusion

This is the end of part 1 of the jets short exercise. If there is spare time remaining, we suggest going through some of the reference material from the first exercise. Otherwise, let's move on to part 2, where we'll talk about jet substructure.